### 1. Input prep

In [1]:
with open('advent-25-0-input.txt') as f:
    keys_locks = f.read().splitlines()

len(keys_locks)

3999

In [71]:
# Let's upload the example to check our code
with open('advent-25-0ex.txt') as f:
    ex = f.read().splitlines()

len(ex)

39

In [74]:
# (1) first convert sequence of symbols into a list
def separate_symbol(liste):
    list_ = []
    for i in range(len(liste)):
        line = list(liste[i])
        list_.append(line)
    return list_

list_ = separate_symbol(keys_locks)
list_ex = separate_symbol(ex)
len(list_), len(list_ex)

(3999, 39)

In [80]:
# (2) let's gather by item (key and lock)
# each item to numpy arrays to ease manipulation with columns
import numpy as np

def separate_items(liste):
    list_2 = list(liste[:7]) # 7 is exclusive
    list_2 = [np.array(list_2)]
    print(f"init size: {len(list_2)}")
    for i in range(7,len(liste),8): # on parcourt chaque ligne
        list_2.append(np.array(list(liste[i+1:i+8])))
    return list_2

list_2 = separate_items(list_)
list_2ex = separate_items(list_ex)
len(list_2), len(list_2ex) # should be 500 (4000 lines / 8) and 5

init size: 1
init size: 1


(500, 5)

In [42]:
len(list_2[0]), len(list_2[1]), list_2[0], list_2[0][6] # sanity checks

(7,
 7,
 array([['#', '#', '#', '#', '#'],
        ['#', '#', '#', '.', '#'],
        ['#', '#', '#', '.', '#'],
        ['.', '#', '#', '.', '#'],
        ['.', '#', '.', '.', '#'],
        ['.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.']], dtype='<U1'),
 array(['.', '.', '.', '.', '.'], dtype='<U1'))

In [49]:
list_2[0][0] == ['#', '#', '#', '#', '#'] # sanity checks

array([ True,  True,  True,  True,  True])

In [ ]:
# let's separate items in lists of keys & locks
def separate_types(liste):
    list_keys = []
    list_locks= []

    for i in range(len(liste)): # on parcourt le nombre d'items dans list_2
        if (liste[i][0] == ['#', '#', '#', '#', '#']).any(): # LOCKS
            list_locks.append(liste[i])
        else:
            list_keys.append(liste[i])
    return list_locks, list_keys

list_locks, list_keys = separate_types(list_2)[0], separate_types(list_2)[1]
list_locks_ex, list_keys_ex = separate_types(list_2ex)[0], separate_types(list_2ex)[1]
len(list_locks), len(list_keys), len(list_locks_ex), len(list_keys_ex) # same number (250) of keys and locks! 
                                                                    # And resp. 2 and 3 for ex

(250, 250, 2, 3)

### 2. Convert to heights

In [56]:
lock_0_col0 = list_locks[0][:,0] # colonne 0 du premier lock
lock_0_col0, type(lock_0_col0)

(array(['#', '#', '#', '.', '.', '.', '.'], dtype='<U1'), numpy.ndarray)

In [57]:
np.count_nonzero(lock_0_col0 == '#') # sanity checks

3

In [ ]:
# Let's convert each lock and key to heights
def count_height(list_items):
    size = len(list_items[0][0]) # same for all locks and keys, nb of columns
    heights = []
    for item in list_items: # on parcourt chaque item
        height_item = []
        for j in range(size):
            height = np.count_nonzero(item[:,j] == '#') - 1 # we start counting at 0 (and there is always at least ONE '#')
            # height = item[:,j].count('#') - 1 # don't use count() which is a method for LISTS (not numpy arrays) 
            height_item.append(height) # liste des heights pour UN ITEM
        heights.append(np.array(height_item)) # np.array ppur faciliter manipulations sur les colonnes
    return heights

list_keys_heights = count_height(list_keys)
list_locks_heights = count_height(list_locks)
list_keys_heights_ex = count_height(list_keys_ex)
list_locks_heights_ex = count_height(list_locks_ex)
len(list_keys_heights), len(list_locks_heights), list_locks_heights_ex, list_keys_heights_ex 
# expected for ex: LOCKS 0,5,3,4,3 and 1,2,0,5,3 || KEYS 5,0,2,1,3 and 4,3,4,0,2 and 3,0,2,0,1 => OK!

(250,
 250,
 [array([0, 5, 3, 4, 3]), array([1, 2, 0, 5, 3])],
 [array([5, 0, 2, 1, 3]), array([4, 3, 4, 0, 2]), array([3, 0, 2, 0, 1])])

In [90]:
type(list_keys_heights[0])

numpy.ndarray

### 3. Calculate the unique lock/key pairs

In [93]:
# For each key we will try every lock and check if there is an overlap

def check_overlap(listeA, listeB): # arguments are list of np.arrays
    liste_sum = []
    count = 0
    # sum keys and locks
    for a_np in listeA:
        for b_np in listeB:
            liste_sum.append(a_np + b_np)
    # check that there is no overlap
    for sum_np in liste_sum:
        if sum_np.max() <= 5:
            count += 1
    return count

check_overlap(list_locks_heights_ex, list_keys_heights_ex) # expected: 3


3

In [94]:
# Le moment de vérité!
check_overlap(list_locks_heights, list_keys_heights)

3287